In [1]:
from thermogravimetricAnalysis import TGA
tga = TGA()

In [2]:
tga.read('angelim', 5, '../examples/assets/T5/angelim.txt')
tga.read('angelim', 10, '../examples/assets/T10/angelim.txt')
tga.read('angelim', 15, '../examples/assets/T15/angelim.txt')
tga.read('angelim', 20, '../examples/assets/T20/angelim.txt')

'angelim at 20 was loaded!'

In [3]:
tga.massLossAndConversions()
tga.dtg('angelim')


In [4]:
for h in [5, 10, 15, 20]:
    id_to_drop = tga.data['angelim'][h][tga.data['angelim'][h]['dtg']>0.3] .index.to_list()
    tga.removeData(id_to_drop, 'angelim', h)

    id_to_drop2 = tga.data['angelim'][h][(tga.data['angelim'][h]['dtg']<=-0.3) & (tga.data['angelim'][h]['temperature']<=200)] .index.to_list()
    tga.removeData(id_to_drop2, 'angelim', h)

In [5]:
tga.smootDataFull()

In [6]:
tga.data['angelim'][5]

,level_0,index,time,temperature,mass,mass_loss,temperature_k,alpha,dtg,dtg_smoothed
0,5,8,8.0,29.659359,10.330079,99.948885,302.809359,0.000511,-0.190406,-0.080895
1,6,9,9.0,29.695827,10.332101,99.970617,302.845827,0.000294,-0.131830,-0.094930
2,10,14,14.0,29.785488,10.331188,99.960804,302.935488,0.000392,0.190111,-0.106317
3,12,16,16.0,29.823046,10.329723,99.945059,302.973046,0.000549,-0.055065,-0.115056
4,14,18,18.0,29.867874,10.327307,99.919094,303.017874,0.000809,-0.283782,-0.121147
...,...,...,...,...,...,...,...,...,...,...
9121,9171,9176,9176.0,809.625793,1.029130,-0.011919,1082.775793,1.000119,0.010213,-0.062849
9122,9172,9177,9177.0,809.699707,1.031586,0.014477,1082.849707,0.999855,0.061497,-0.073266
9123,9173,9178,9178.0,809.791992,1.030081,-0.001698,1082.941992,1.000017,0.098441,-0.084889
9124,9174,9179,9179.0,809.885742,1.033290,0.032790,1083.035742,0.999672,0.009841,-0.097716


In [7]:
import numpy as np

In [8]:
def intermediateFWOCalculation(sample, alpha_min=0.1, alpha_max=0.8, alpha_spacing=0.1, alpha_array=None):
    
    if alpha_array is None:
        # multiply 10 and divide to avoid floating point error
        alpha_range = np.arange(10*alpha_min, 10*(alpha_max+alpha_spacing), 10*alpha_spacing)/10
    else:
        alpha_range = alpha_array

    data = tga.data[sample]
    heatingRate = tga.sampleHeatingRates[sample]
    result = []
    alpha_result = []
    for alpha in alpha_range:

        for hr in heatingRate:
            id = data[hr][data[hr]['alpha'] < alpha]['index'].idxmax()
            temperature = data[hr].iloc[id]['temperature_k']
            time = data[hr].iloc[id]['time']
            alpha_result.append( (alpha, 1/temperature, temperature, time) )
            
        result.append( alpha_result )
        alpha_result = []

    return result

In [9]:
from kinetics import Kinetics

In [10]:
k = Kinetics()
k.loadData(tga.data['angelim'], tga.sampleHeatingRates['angelim'])



In [15]:
k.FWO(alpha_min=0.05, alpha_max=0.95, alpha_spacing=0.05)

,alpha,Slope,R,R2,Intercept,Poison,Std_Error,Ea,Temperature_Kelvin,Temperature_Celsius,Time
0,0.05,-2593.594260,-0.582769,0.339619,8.417009,0.417231,2557.336378,47.186745,360.022223,86.872223,410.0
1,0.10,-2407.457149,-0.221844,0.049215,5.491412,0.778156,7482.348665,43.800246,551.221350,278.071350,1546.0
2,0.15,-7518.240641,-0.549181,0.301600,14.284583,0.450819,8089.805979,136.783656,574.836676,301.686676,1684.0
3,0.20,-7667.040491,-0.501526,0.251529,14.150499,0.498474,9352.049337,139.490857,591.964453,318.814453,1784.0
4,0.25,-8001.807208,-0.498870,0.248872,14.386888,0.501130,9829.750689,145.581459,606.717322,333.567322,1869.0
5,0.30,-8686.344437,-0.517561,0.267869,15.236950,0.482439,10154.419076,158.035637,618.912909,345.762909,1939.0
6,0.35,-10120.770302,-0.581346,0.337963,17.309290,0.418654,10016.257711,184.132968,628.647986,355.497986,1995.0
7,0.40,-11142.039595,-0.640629,0.410405,18.705009,0.359371,9443.225478,202.713505,637.028540,363.878540,2043.0
8,0.45,-12718.584051,-0.736943,0.543085,20.987038,0.263057,8249.113657,231.396481,643.126074,369.976074,2080.0
9,0.50,-13506.259630,-0.806159,0.649892,22.034875,0.193841,7009.723152,245.727113,648.129126,374.979126,2109.0


In [12]:
tga.data['angelim'][5].iloc[2793]['temperature_k']

535.370551

In [13]:
tga.sampleHeatingRates

{'angelim': [5, 10, 15, 20]}

In [14]:
np.linspace(0, 1, num=9)

array([0.   , 0.125, 0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   ])